In [ ]:
import util
from test_framework.address import program_to_witness
from test_framework.key import generate_key_pair, generate_schnorr_nonce
from test_framework.messages import CScriptWitness, CTxInWitness
from test_framework.musig import aggregate_musig_signatures, aggregate_schnorr_nonces, generate_musig_key, sign_musig
from test_framework.script import SIGHASH_ALL_TAPROOT, TaprootSignatureHash

# 2.1 Taproot Outputs

* Part 1: Generating a segwit v1 output address
* Part 2: Sending funds from the Bitcoin Core wallet
* Part 3: Constructing a transaction to spend the segwit v1 output
    * Spending a segwit v1 output with a MuSig public key

In this chapter, we introduce segwit v1 outputs, which are defined in [bip-taproot](https://github.com/bitcoinops/bips/blob/v0.1/bip-taproot.mediawiki). Segwit v1 outputs can be spent in two ways:

* **Key path** spending, which treats the witness program as a public key, and permits spending using a signature from that public key.
* **Script path** spending, which allows a pre-committed script to be used to spend the output. Script path spending will be fully described in chapters 2.2, 2.3 and 2.4.

By using the MuSig pubkey and signature aggregation protocol described in chapter 1.2, key path spending can be used to encumber an output to an n-of-n multisig policy in a way that is indistinguishable from a single-key output and spend.

The first half of this chapter shows an example of sending funds to a segwit v1 address using the Bitcoin Core wallet, and then manually constructing a transaction that spends that output using the new bip-taproot key path spending rules.

There is then a coding exercise to send funds to a MuSig aggregate pubkey address, and then construct a transaction that spends from that address using the bip-taproot key path spending rules, following the same steps as the example.

## Part 1. Generating a segwit v1 output

Segwit v1 follows the same output script pattern as segwit v0:

* Segwit output: **`[1B Version]` `[segwit program]`**
* Segwit v0 output: **`[00]` `[20-Byte public key digest]`** (P2WPKH) or **`[00]` `[32-Byte script digest]`** (P2WSH)
* Segwit v1 output: **`[01]` `[33-Byte public key]`**

The 33-Byte public key encoding is similar to that of legacy compressed pubkeys, but with a different oddness byte.

* Y-coordinate: even - **`[00]`** or odd - **`[01]`**
* X-coordinate: **`[32B x-coordinate]`**
* 33 byte pubkey encoding: **`[1B oddness] [32B x-coordinate]`**

### Spending a segwit v1 output with the key path

Unlike segwit v0 outputs, v1 outputs look the same for script or key paths(unlike v0 which separates into P2WPKH and P2WSH). In this chapter we will focus on spending the key path.

![test](images/segwit_version1_0.jpg)

The output can be spent along the **key path** by providing a valid signature for the pubkey in the output's scriptPubKey. The spending witness is simply **`[sig]`**.

The output can be spent along the **script path** if public key was tweaked with a valid taproot. See chapters 2.2 and 2.3 for further details.

#### 2.1.1 Example: Constructing a segwit v1 output

In this example, we construct segwit v1 output for spending along the key path. We generate a key pair, encode the public key using the bip-schnorr and bip-taproot pubkey encoding rules, and then encode the witness version and witness program to a bech32 address.

In [ ]:
# Key pair generation
privkey, pubkey = generate_key_pair()
print("Pubkey is {}\n".format(pubkey.get_bytes().hex()))

# Create witness program ([1B oddness] [32B x-coordinate])
pubkey_data = pubkey.get_bytes()
program = bytes([pubkey_data[0] & 1]) + pubkey_data[1:]
print("Witness program is {}\n".format(program.hex()))

# Create (regtest) bech32 address
version = 0x01
address = program_to_witness(version, program)
print("bech32 address is {}".format(address))

## Part 2. Sending funds from the Bitcoin Core wallet

Next, we send funds to the segwit v1 address that we just generated. We'll create send the funds from a Bitcoin Core wallet, which is able to send outputs to segwit v1 addresses.

#### Example 2.1.2: Start Bitcoin Core node and send coins to the taproot address

Only run setup once, or after a clean shutdown.

In [ ]:
# Start node
test = util.TestWrapper()
test.setup()
node = test.nodes[0]

# Generate coins and create an output
tx = node.generate_and_send_coins(address)
print("Transaction {}, output 0\nsent to {}\n".format(tx.hash, address))

## Part 3. Constructing a transaction to spend the segwit v1 output

We are now going to manually contruct, sign and broadcast a transaction which spends the segwit v1 output.

To do that we create a `CTransaction` object and populate the data members:

 * `nVersion`
 * `nLocktime`  
 * `tx_vin` (list of `CTxIn` objects)
 * `tx_vout` (list of `CTxOut` objects)
 * `tx.wit.vtxinwit` (list of `CTxInWitness` objects)

#### Example 2.1.3: Construct `CTransaction` and populate fields

We use the `create_spending_transaction(node, txid)` convenience function.

In [ ]:
# Create a spending transaction
spending_tx = test.create_spending_transaction(tx.hash)
print("Spending transaction:\n{}".format(spending_tx))

#### Example 2.1.4: Sign the transaction with a schnorr signature

bip-taproot defines the following sighash flags:
* Legacy sighash flags:
  * `0x01` - **SIGHASH_ALL**
  * `0x02` - **SIGHASH_NONE**
  * `0x03` - **SIGHASH_SINGLE**
  * `0x81` - **SIGHASH_ALL | SIGHASH_ANYONECANPAY**
  * `0x82` - **SIGHASH_NONE | SIGHASH_ANYONECANPAY**
  * `0x83` - **SIGHASH_SINGLE | SIGHASH_ANYONECANPAY**
* New sighash flag:
  * `0x00` - **SIGHASH_ALL_TAPROOT** same semantics `0x01` **SIGHASH_ALL**

Append the sighash flag to the signature `[R_x, s]` with the sighash byte if not `0x00`.

In [ ]:
# Generate the taproot signature hash for signing
# SIGHASH_ALL_TAPROOT is 0x00
sighash = TaprootSignatureHash(spending_tx, [tx.vout[0]], SIGHASH_ALL_TAPROOT, input_index=0)
 
# All schnorr sighashes except SIGHASH_ALL_TAPROOT require
# the hash_type appended to the end of signature
sig = privkey.sign_schnorr(sighash)

print("Signature: {}".format(sig.hex()))

#### Example 2.1.5: Add the witness and test acceptance of the transaction

In [ ]:
# Construct transaction witness
witness = CScriptWitness()
witness.stack.append(sig)
witness_in = CTxInWitness()
witness_in.scriptWitness = witness

# vtxinwit is a list of the witness data (signatures, etc.)
spending_tx.wit.vtxinwit.append(witness_in)

print("Spending transaction:\n{}\n".format(spending_tx))
 
# Test mempool acceptance
node.test_transaction(spending_tx)
print("Success!")

#### Example 2.1.6: Shutdown the TestWrapper (and all bitcoind instances)

In [ ]:
test.shutdown()

### Spending a segwit v1 output with a MuSig public key

In this exercise, we'll use the Bitcoin Core wallet to create a transaction that sends to a 2-of-2 musig aggregate pubkey (segwit v1 address).

We'll then manually create a transaction that spends the MuSig output and sends an output back to the Bitcoin Core wallet. We'll use Bitcoin Core's `testmempoolaccept()` RPC method to verify that the transaction is valid.

#### 2.1.7 _Programming Exercise:_ Generate segwit v1 addresses for a 2-of-2 MuSig aggregate pubkey

In this exercise, we create a 2-of-2 aggregate MuSig public key

In [ ]:
# Generate individual key pairs
privkey1, pubkey1 = generate_key_pair()
privkey2, pubkey2 = generate_key_pair()

# Generate a 2-of-2 aggregate MuSig key
# Method: generate_musig_key(ECPubKey_list)
c_map, agg_pubkey =  # TODO: implement

# Multiply individual keys with challenges
privkey1_c =  # TODO: implement
privkey2_c =  # TODO: implement
pubkey1_c =  # TODO: implement
pubkey2_c =  # TODO: implement

# Create a segwit v1 address for the MuSig aggregate pubkey
# Method: address = program_to_witness(version_int, program_bytes)
program_musig =  # TODO: implement
address_musig =  # TODO: implement
print("2-of-2 musig: ", address_musig)

#### Example 2.1.8: Create a transaction in the Bitcoin Core wallet sending outputs to the segwit v1 addresses

In [ ]:
test = util.TestWrapper()
test.setup()
node = test.nodes[0]

# Generate coins and create an output
tx = node.generate_and_send_coins(address_musig)
print("Transaction {}, output 0\nsent to {}\n".format(tx.hash, address_musig))

#### Example 2.1.9 : Construct CTransaction and populate fields

In [ ]:
# Create a spending transaction
spending_tx = test.create_spending_transaction(tx.hash)
print("Spending transaction:\n{}".format(spending_tx))

#### 2.1.10 _Programming Exercise:_ Create a valid bip-schnorr signature for the MuSig aggregate pubkey

In this exercise, we create a signature for the aggregate pubkey, add it to the witness, and then test that the transaction is accepted by the mempool.

In [ ]:
# Create sighash for ALL (0x00)
sighash_musig = TaprootSignatureHash(spending_tx, [tx.vout[0]], SIGHASH_ALL_TAPROOT, input_index=0)

# Generate individual nonces for participants and an aggregate nonce point
# Remember to negate the individual nonces if necessary
# Method: generate_schnorr_nonce()
# Method: aggregate_schnorr_nonces(nonce_list)
nonce1 = # TODO: implement
nonce2 = # TODO: implement
R_agg, negated =  # TODO: implement

# Create an aggregate signature
# Method: sign_musig(privkey, nonce, R_agg, agg_pubkey, sighash_musig)
# Method: aggregate_musig_signatures(partial_signature_list, R_agg)
s1 = # TODO: implement
s2 = # TODO: implement
sig_agg =  # TODO:implement
print("Aggregate signature is {}\n".format(sig_agg.hex()))

# Construct transaction witness
witness = CScriptWitness()
witness.stack.append(  # TODO: implement
witness_in = CTxInWitness()
witness_in.scriptWitness = witness
spending_tx.wit.vtxinwit.append(witness_in)

# Test mempool acceptance
assert node.test_transaction(spending_tx)
print("Success!")

#### Example 2.1.11: Shutdown the TestWrapper (and all bitcoind instances)

In [ ]:
test.shutdown()

**Congratulations!** In this chapter, you have:

- Learned how to create a segwit v1 output and derive its bech32 address.
- Started a Bitcoin Core full node (in regtest mode), generated 101 blocks and sent a transaction output to the segwit v1 output.
- Constructed a transaction that spends the segwit v1 output back to the wallet using the key path.
- Repeated the above steps for a MuSig aggregate pubkey and signature.